In [11]:
using Flux, Flux.Data.MNIST, Statistics
using Flux: @epochs, mse, onehotbatch, onecold
using Random
using Base.Iterators: partition
using BSON: @load, @save


imgs = MNIST.images(:train)
train_X = hcat(float.(vec.(imgs))...)
labels = MNIST.labels(:train)
train_Y = onehotbatch(labels,0:9)

batchsize = 64
train_dataset = ([(train_X[:,batch] ,train_Y[:,batch]) for batch in partition(1:size(train_Y)[2],batchsize)])

model = Chain(
  Dense(28^2, 32, relu),
  Dense(32, 32, relu),
  Dense(32, 10),
  softmax)

loss(x, y) = mse(model(x), y) 
opt = ADAM(params(model))

@epochs 100 Flux.train!(loss, train_dataset, opt)

# モデルの保存
pretrained = model |> cpu
weights = Tracker.data.(params(pretrained))
@save "pretrained.bson" pretrained
@save "weights.bson" weights
println("Finished to train")

┌ Info: Epoch 1
└ @ Main /Users/oshita/.julia/packages/Flux/oN61x/src/optimise/train.jl:93
┌ Info: Epoch 2
└ @ Main /Users/oshita/.julia/packages/Flux/oN61x/src/optimise/train.jl:93
┌ Info: Epoch 3
└ @ Main /Users/oshita/.julia/packages/Flux/oN61x/src/optimise/train.jl:93
┌ Info: Epoch 4
└ @ Main /Users/oshita/.julia/packages/Flux/oN61x/src/optimise/train.jl:93
┌ Info: Epoch 5
└ @ Main /Users/oshita/.julia/packages/Flux/oN61x/src/optimise/train.jl:93
┌ Info: Epoch 6
└ @ Main /Users/oshita/.julia/packages/Flux/oN61x/src/optimise/train.jl:93
┌ Info: Epoch 7
└ @ Main /Users/oshita/.julia/packages/Flux/oN61x/src/optimise/train.jl:93
┌ Info: Epoch 8
└ @ Main /Users/oshita/.julia/packages/Flux/oN61x/src/optimise/train.jl:93
┌ Info: Epoch 9
└ @ Main /Users/oshita/.julia/packages/Flux/oN61x/src/optimise/train.jl:93
┌ Info: Epoch 10
└ @ Main /Users/oshita/.julia/packages/Flux/oN61x/src/optimise/train.jl:93
┌ Info: Epoch 11
└ @ Main /Users/oshita/.julia/packages/Flux/oN61x/src/optimise/train.jl:

Finished to train


In [12]:
function prepare_dataset(;train=true)
    train_or_test = ifelse(train,:train,:test)
    imgs = MNIST.images(train_or_test)
    X = hcat(float.(vec.(imgs))...)
    labels = MNIST.labels(train_or_test)
    Y = onehotbatch(labels,0:9)
    return X, Y
end

function split_dataset_random(X, Y)
    divide_ratio=0.9
    shuffled_indices = shuffle(1:size(Y)[2])
    divide_idx = round(Int,0.9*length(shuffled_indices))
    train_indices = shuffled_indices[1:divide_idx]
    val_indices = shuffled_indices[divide_idx:end]
    train_X = X[:,train_indices]
    train_Y = Y[:,train_indices]
    val_X = X[:,val_indices]
    val_Y = Y[:,val_indices]
    return train_X, train_Y, val_X, val_Y
end

function predict()
    println("Start to evaluate testset")
    println("loading pretrained model")
    @load "pretrained.bson" pretrained
    model = pretrained |> cpu
    accuracy(x, y) = mean(onecold(model(x)) .== onecold(y))
    println("prepare dataset")
    X, Y = prepare_dataset(train=false)
    X = X |> cpu
    Y = Y |> cpu
    @show accuracy(X, Y)
    println("Done")
end

predict (generic function with 1 method)

In [13]:
predict()

Start to evaluate testset
loading pretrained model
prepare dataset
accuracy(X, Y) = 0.9691
Done


In [15]:
x = rand(10)

10-element Array{Float64,1}:
 0.9380907398527807 
 0.9485765624748486 
 0.21334805456119743
 0.2790950070461595 
 0.6183961892425358 
 0.6946523945701855 
 0.5817717182194733 
 0.14715678256368392
 0.9176965621324424 
 0.8508407683224619 

In [16]:
@code_js identity(x)
let model = (function () {
  let math = tf;
  function model(kinkajou) {
    return kinkajou;
  };
  model.weights = [];
  return model;
})();
flux.fetchWeights("pretrained.bson").then((function (ws) {
  return model.weights = ws;
}));

LoadError: UndefVarError: @code_js not defined